In [87]:
import sympy as sp

In [88]:
x, dx = sp.symbols('x \\dot{x}')
m, g = sp.symbols('m g')

In [89]:
KE = m * dx**2 / 2
PE = m*g*x
L = KE - PE 

In [90]:
L

\dot{x}**2*m/2 - g*m*x

In [104]:
def time_derivative(f, variables):
    if f.is_Add:
        return sp.Add(*[ time_derivative(a, variables) for a in f.args ])
    
    if f.is_Mul:
        return sp.Add(*[ f / a * time_derivative(a, variables) for a in f.args ])
    
    if f.is_Pow:
        return f.exp * f.base ** (f.exp - 1) * time_derivative(f.base, variables)
    
    if f.is_Symbol:
        if str(f).startswith('\\dot{'):
            g = sp.Symbol(str(f)[5:-1])
            if g in variables:
                return sp.Symbol(f'\\ddot{{{g}}}')
        if f in variables:
            return sp.Symbol(f'\\dot{{{f}}}')
        else:
            return 0
        
    if f.is_Number:
        return 0
    
    print(f'Unsupported: {f}')

In [105]:
def euler_lagrange(L, xs):
    eqs = []
    for x in xs:
        v = sp.Symbol(f'\\dot{{{x}}}')
        dL_dx = L.diff(x)
        dL_dv = L.diff(v)
        ddL_dv_dt = time_derivative(dL_dv, [ x ])
        eqs.append(dL_dx - ddL_dv_dt)

    return eqs

In [121]:
def linear_system(L, xs):
    eqs = euler_lagrange(L, xs)
    
    accs = [ sp.Symbol(f'\\ddot{{{x}}}') for x in xs ]
    
    polys = [ sp.Poly(eq, accs) for eq in eqs ]
        
    A = sp.Matrix([ [ t[1] for t in poly.terms()[0:-1] ] for poly in polys ])
    b = sp.Matrix([[ poly.terms()[-1][1] ] for poly in polys ])

    return A, b

In [123]:
A, b = linear_system(L, [ x ])

In [125]:
A

Matrix([[-m]])

In [126]:
b

Matrix([[-g*m]])

In [128]:
sp.linsolve((A, b))

FiniteSet((g,))